In [6]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from nilearn.image import concat_imgs, mean_img
from nilearn.plotting import plot_stat_map, plot_design_matrix, plot_contrast_matrix
from nilearn.glm.first_level import FirstLevelModel
from nilearn.reporting import get_clusters_table
from nilearn.glm.thresholding import threshold_stats_img

from bids import BIDSLayout

In [7]:
# ACNets project folder (to access data and resources)
project_dir = Path('.')

In [8]:
layout = BIDSLayout(project_dir / 'data/julia2018', derivatives=True)

task = 'attention'
sessions = ['1','2']
# subjects = ['NVGP01']
subjects = layout.get_subject()

fmri_images = layout.get(task='attention', desc='preproc', suffix='bold', scope='derivatives',
                         extension='nii.gz', return_type='filename')


mask_images = layout.get(task='attention', desc='brain', suffix='mask', scope='derivatives',
                         extension='nii.gz', return_type='filename')

confounds_files = layout.get(task='attention', desc='confounds', suffix='timeseries',
                             scope='derivatives', extension='tsv', return_type='filename')

events = layout.get(task='attention', desc='preproc', suffix='events', scope='derivatives',
                    extension='tsv', return_type='filename')

confounds_cols = ['trans_x', 'trans_y', 'trans_z',
                  'rot_x', 'rot_y', 'rot_z',
                  'global_signal',
                  'a_comp_cor_00', 'a_comp_cor_01']

TR = layout.get_tr()


BIDSValidationError: 'dataset_description.json' is missing from project root. Every valid BIDS dataset must have this file.
Example contents of 'dataset_description.json': 
{"Name": "Example dataset", "BIDSVersion": "1.0.2"}

In [ ]:
# %% load data
fmri_image = concat_imgs(fmri_images)
mean_image = mean_img(fmri_image)

# apply Gaussian kernel of 8mm (foecker was 5mm) and band-pass filtering
# run-specific intercept
# convolve with canonical HRF
# GLM predictor = target onset of different trial types
# additional regressors = error, post-error, timeout, and outlier trials

# combine both sessions as foecker2018 found no relevant differences acorss sessions.
# analyze all trials (correct and incorrect)
# foecker2018 also used SOA as regressor
# Emiliano used 90-ROIs from Shirer for resting state https://findlab.stanford.edu/functional_ROIs.html
# a more comperhensive atlas would be willard which contains 499 ROIs
# Emiliano used filtering of ([0.008 0.09]) 

# planned contrasts:
#   - invalid vs valid
#   - left vs right ( * valid vs invalid)
#   - interaction of cue vs target_position
#   - distractors????

events = pd.read_table(events)
print(events)
# TODO filter events


In [ ]:
# fit GLM

fmri_glm = FirstLevelModel(t_r=TR,
                           noise_model='ar1',
                           standardize=False,
                           hrf_model='spm',
                           drift_model='cosine',
                           high_pass=.01)

fmri_glm = fmri_glm.fit(fmri_images, events)


In [ ]:
# plot design matrix
# row=time, col=predictors
design_matrix = fmri_glm.design_matrices_[0]

plot_design_matrix(design_matrix)
plt.show()

plt.plot(design_matrix['cue'])
plt.xlabel('scan')
plt.title('Expected response')
plt.show()

In [ ]:
# calculate contrast matrix
conditions = {
    'valid': np.array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
    'invalid': np.array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
}

valid_minus_invalid = conditions['valid'] - conditions['invalid']

In [ ]:
# plot contrast matrix

plot_contrast_matrix(valid_minus_invalid, design_matrix=design_matrix)

In [ ]:
# effect size map, z map, and plot them
eff_map = fmri_glm.compute_contrast(valid_minus_invalid, output_type='effect_size')

z_map = fmri_glm.compute_contrast(valid_minus_invalid, output_type='z_score')

plot_stat_map(z_map, bg_img=mean_img, threshold=3.0,
              display_mode='z', cut_coords=3, black_bg=True,
              title='Active minus Rest (Z>3)')
plt.show()

In [ ]:
# print clusters info and coordinates
clean_map, threshold = threshold_stats_img(z_map, alpha=.05, height_control='fdr', cluster_threshold=10)

table = get_clusters_table(z_map, stat_threshold=threshold, cluster_threshold=20)
print(table)